In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', 700)
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore") 

In [2]:
df_train = pd.read_csv('TRAIN (1).csv')
df_train['order_date'] = pd.to_datetime(df_train['order_date'], format = '%m/%d/%y')
df_train['evsd'] = pd.to_datetime(df_train['evsd'], format = '%m/%d/%y')
df_train['tag'] = 'train'

print(df_train.shape)
df_train.head()

(259287, 14)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag
0,1,JFK2,540,30718,67271,2022-11-02,2022-11-14,AFT Lite,54,54,54,4,9,train
1,1,JFK2,540,30718,125389,2022-11-20,2022-12-01,AFT Lite,60,60,60,5,3,train
2,1,JFK2,648,16750,11165,2022-09-30,2022-10-03,AFT Lite,6,6,6,1,2,train
3,1,JFK2,648,16750,27008,2022-10-14,2022-10-17,AFT Lite,6,6,6,2,2,train
4,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train


In [3]:
# Create a new column that combines the values of the existing columns
df_train['unique_id'] = df_train.apply(lambda row: f"{row['warehouse_id']}_{row['d_id']}_{row['isbn_id']}_{row['order_id']}", axis=1)

# Drop duplicates based on the new column
df_unique = df_train.drop_duplicates(subset=['unique_id'])

df_unique.shape, df_train.shape

((101777, 15), (259287, 15))

In [4]:
df_train.head()

,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id
0,1,JFK2,540,30718,67271,2022-11-02,2022-11-14,AFT Lite,54,54,54,4,9,train,JFK2_540_30718_67271
1,1,JFK2,540,30718,125389,2022-11-20,2022-12-01,AFT Lite,60,60,60,5,3,train,JFK2_540_30718_125389
2,1,JFK2,648,16750,11165,2022-09-30,2022-10-03,AFT Lite,6,6,6,1,2,train,JFK2_648_16750_11165
3,1,JFK2,648,16750,27008,2022-10-14,2022-10-17,AFT Lite,6,6,6,2,2,train,JFK2_648_16750_27008
4,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598


In [5]:
df_train['visibility'].dtype

dtype('int64')

In [6]:
# How long does order usually take to deliver?
# df_unique['order_delivery_days'] = (df_unique['evsd'] - df_unique['order_date']).dt.days
# df_test['order_delivery_days'] = (df_test['evsd'] - df_test['order_date']).dt.days

df_unique['consider'] =   np.where((df_unique['evsd'] - df_unique['order_date']).dt.days  == df_unique['visibility'], True, False)




In [7]:
df_unique['consider'].value_counts(dropna= False)

consider
True     54085
False    47692
Name: count, dtype: int64

In [10]:
df_test = pd.read_csv('Test.csv', encoding = 'latin1')
# df_test.columns = ['LOCATION', 'd_id', 'isbn_id', 'order_id_1', 'order_date', 'evsd',
#        'row_group_ind', 'visibility', 'quantity_ordered', 'quantity_submitted',
#        'quantity_received']
df_test['order_date'] = pd.to_datetime(df_test['order_date'], format = '%m/%d/%y')
df_test['evsd'] = pd.to_datetime(df_test['evsd'], format = '%m/%d/%y')
df_test['tag'] = 'test'
print(df_test.shape)
df_test.head()

(98309, 14)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test
2,1,JFK2,509,19749,119943,2022-12-15,2022-12-17,AFT Lite,12,12,12,3,1,test
3,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test
4,1,JFK2,540,20260,117854,2022-12-26,2023-01-04,AFT Lite,60,60,60,1,8,test


In [17]:
# Apply same filter on Test Data too
# Create a new column that combines the values of the existing columns
df_test['unique_id'] = df_test.apply(lambda row: f"{row['warehouse_id']}_{row['d_id']}_{row['isbn_id']}_{row['order_id']}", axis=1)
print(df_test.shape)
# Drop duplicates based on the new column
df_test_n = df_test.drop_duplicates(subset=['unique_id'])

df_test_n.shape

(98309, 16)


(40996, 16)

In [18]:
df_test_n['consider'] =   np.where((df_test_n['evsd'] - df_test_n['order_date']).dt.days  == df_test_n['visibility'], True, False)
df_test_n['consider'].value_counts(dropna= False)

consider
True     22406
False    18590
Name: count, dtype: int64

## Final DATA : ``df_unique`` and ``df_test``

In [19]:
df_unique = df_unique.loc[df_unique['consider'] == True].reset_index(drop=True)
df_test_n = df_test_n.loc[df_test_n['consider'] == True].reset_index(drop=True)


In [20]:
df_test_n.shape

(22406, 16)

In [21]:
# Feature Engineering
for col in ['d_id', 'isbn_id', 'order_id']:
  df_unique[col] = df_unique[col].astype(int).astype(str)
  df_test[col] = df_test[col].astype(int).astype(str)
  df_test_n[col] = df_test_n[col].astype(int).astype(str)

print(df_unique.shape)
display(df_unique.head(2))

print(df_test.shape)
df_test.head(2)

(54085, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True


(98309, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test,JFK2_684_674_45573,False
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True


In [24]:
# Feature Engineering
for col in ['d_id', 'isbn_id', 'order_id']:
  df_unique[col] = df_unique[col].astype(int).astype(str)
  df_test[col] = df_test[col].astype(int).astype(str)
  df_test_n[col] = df_test_n[col].astype(int).astype(str)

print(df_unique.shape)
display(df_unique.head(2))

print(df_test.shape)
df_test.head(2)

(54085, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True


(98309, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test,JFK2_684_674_45573,False
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True


In [25]:
df_unique['row_group_ind'] = df_unique['row_group_ind'].astype(str)
df_test['row_group_ind'] = df_test['row_group_ind'].astype(str)
df_test_n['row_group_ind'] = df_test_n['row_group_ind'].astype(str)

var_dict_train_ ={}

for col in ['row_group_ind', 'd_id']:
  var_dict_train_[col] = [x for x in df_unique[col].value_counts().sort_values(ascending=False).index]


display(var_dict_train_)

{'row_group_ind': ['3', '4', '2', '5', '1', '6', '7', '0'],
 'd_id': ['509',
  '7',
  '110',
  '584',
  '648',
  '566',
  '1',
  '643',
  '454',
  '639',
  '684',
  '410',
  '540',
  '174',
  '472',
  '763',
  '830',
  '136',
  '711',
  '343',
  '524',
  '652',
  '18',
  '373',
  '13',
  '393',
  '845',
  '218',
  '302',
  '241',
  '127',
  '613']}

In [26]:
# Define a funtion to peform one hot encoding for the top 10 most frequent categories of Categorical Features
def one_hot_top10(df,feature,top10_categories):
    for category in top10_categories:
        df[feature+'_'+category]=np.where(df[feature]==category,1,0)

In [27]:
for key,val in var_dict_train_.items():
  one_hot_top10(df_unique,key,val)

for key,val in var_dict_train_.items():
  one_hot_top10(df_test,key,val)

for key,val in var_dict_train_.items():
  one_hot_top10(df_test_n,key,val)

display(df_unique.head())
display(df_test.head())

,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,JFK2,7,8748,27330,2022-11-05,2022-11-06,AFT Lite,0,24,0,6,1,train,JFK2_7_8748_27330,True,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,JFK2,7,17780,122956,2022-11-12,2022-11-13,AFT Lite,36,36,36,3,1,train,JFK2_7_17780_122956,True,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,JFK2,7,27826,175768,2022-10-11,2022-10-12,AFT Lite,24,24,24,5,1,train,JFK2_7_27826_175768,True,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test,JFK2_684_674_45573,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,JFK2,509,19749,119943,2022-12-15,2022-12-17,AFT Lite,12,12,12,3,1,test,JFK2_509_19749_119943,False,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,JFK2,540,20260,117854,2022-12-26,2023-01-04,AFT Lite,60,60,60,1,8,test,JFK2_540_20260_117854,False,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:

df_unique['order_day'] = df_unique['order_date'].dt.day
df_unique['order_dayofweek'] = df_unique['order_date'].dt.dayofweek
df_unique['order_dayofyear'] = df_unique['order_date'].dt.dayofyear
df_unique['order_month'] = df_unique['order_date'].dt.month

df_unique['delivered_day'] = df_unique['evsd'].dt.day
df_unique['delivered_dayofweek'] = df_unique['evsd'].dt.dayofweek
df_unique['delivered_dayofyear'] = df_unique['evsd'].dt.dayofyear
df_unique['delivered_month'] = df_unique['evsd'].dt.month

df_unique['order_month_sin'] = np.sin(2 * np.pi * df_unique['order_month'] / 12)
df_unique['order_month_cos'] = np.cos(2 * np.pi * df_unique['order_month'] / 12)

df_unique['order_dayofweek_sin'] = np.sin(2 * np.pi * df_unique['order_dayofweek'] / 7)
df_unique['order_dayofweek_cos'] = np.cos(2 * np.pi * df_unique['order_dayofweek'] / 7)

df_unique['delivered_month_sin'] = np.sin(2 * np.pi * df_unique['delivered_month'] / 12)
df_unique['delivered_month_cos'] = np.cos(2 * np.pi * df_unique['delivered_month'] / 12)

df_unique['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_unique['delivered_dayofweek'] / 7)
df_unique['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_unique['delivered_dayofweek'] / 7)

df_unique.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_unique.head())


df_test['order_day'] = df_test['order_date'].dt.day
df_test['order_dayofweek'] = df_test['order_date'].dt.dayofweek
df_test['order_dayofyear'] = df_test['order_date'].dt.dayofyear
df_test['order_month'] = df_test['order_date'].dt.month

df_test['delivered_day'] = df_test['evsd'].dt.day
df_test['delivered_dayofweek'] = df_test['evsd'].dt.dayofweek
df_test['delivered_dayofyear'] = df_test['evsd'].dt.dayofyear
df_test['delivered_month'] = df_test['evsd'].dt.month

df_test['order_month_sin'] = np.sin(2 * np.pi * df_test['order_month'] / 12)
df_test['order_month_cos'] = np.cos(2 * np.pi * df_test['order_month'] / 12)

df_test['order_dayofweek_sin'] = np.sin(2 * np.pi * df_test['order_dayofweek'] / 7)
df_test['order_dayofweek_cos'] = np.cos(2 * np.pi * df_test['order_dayofweek'] / 7)

df_test['delivered_month_sin'] = np.sin(2 * np.pi * df_test['delivered_month'] / 12)
df_test['delivered_month_cos'] = np.cos(2 * np.pi * df_test['delivered_month'] / 12)

df_test['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_test['delivered_dayofweek'] / 7)
df_test['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_test['delivered_dayofweek'] / 7)

df_test.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_test.head())


df_test_n['order_day'] = df_test_n['order_date'].dt.day
df_test_n['order_dayofweek'] = df_test_n['order_date'].dt.dayofweek
df_test_n['order_dayofyear'] = df_test_n['order_date'].dt.dayofyear
df_test_n['order_month'] = df_test_n['order_date'].dt.month

df_test_n['delivered_day'] = df_test_n['evsd'].dt.day
df_test_n['delivered_dayofweek'] = df_test_n['evsd'].dt.dayofweek
df_test_n['delivered_dayofyear'] = df_test_n['evsd'].dt.dayofyear
df_test_n['delivered_month'] = df_test_n['evsd'].dt.month

df_test_n['order_month_sin'] = np.sin(2 * np.pi * df_test_n['order_month'] / 12)
df_test_n['order_month_cos'] = np.cos(2 * np.pi * df_test_n['order_month'] / 12)

df_test_n['order_dayofweek_sin'] = np.sin(2 * np.pi * df_test_n['order_dayofweek'] / 7)
df_test_n['order_dayofweek_cos'] = np.cos(2 * np.pi * df_test_n['order_dayofweek'] / 7)

df_test_n['delivered_month_sin'] = np.sin(2 * np.pi * df_test_n['delivered_month'] / 12)
df_test_n['delivered_month_cos'] = np.cos(2 * np.pi * df_test_n['delivered_month'] / 12)

df_test_n['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_test_n['delivered_dayofweek'] / 7)
df_test_n['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_test_n['delivered_dayofweek'] / 7)

df_test_n.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_test_n.head())


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,1,319,16,2,320,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,305,2,2,306,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521
2,1,JFK2,7,8748,27330,2022-11-05,2022-11-06,AFT Lite,0,24,0,6,1,train,JFK2_7_8748_27330,True,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,309,6,6,310,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490
3,1,JFK2,7,17780,122956,2022-11-12,2022-11-13,AFT Lite,36,36,36,3,1,train,JFK2_7_17780_122956,True,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,5,316,13,6,317,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490
4,1,JFK2,7,27826,175768,2022-10-11,2022-10-12,AFT Lite,24,24,24,5,1,train,JFK2_7_27826_175768,True,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,284,12,2,285,-0.866025,0.500000,0.781831,0.623490,-0.866025,0.500000,0.974928,-0.222521


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test,JFK2_684_674_45573,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,3,349,19,0,353,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.000000,0.000000,1.000000
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,1,361,30,4,364,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.000000,-0.433884,-0.900969
2,1,JFK2,509,19749,119943,2022-12-15,2022-12-17,AFT Lite,12,12,12,3,1,test,JFK2_509_19749_119943,False,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,3,349,17,5,351,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.000000,-0.974928,-0.222521
3,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,1,354,23,4,357,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.000000,-0.433884,-0.900969
4,1,JFK2,540,20260,117854,2022-12-26,2023-01-04,AFT Lite,60,60,60,1,8,test,JFK2_540_20260_117854,False,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,360,4,2,4,-2.449294e-16,1.0,0.000000,1.000000,5.000000e-01,0.866025,0.974928,-0.222521


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,1,361,30,4,364,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969
1,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,1,354,23,4,357,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969
2,1,JFK2,540,32203,112020,2022-12-07,2022-12-16,AFT Lite,18,18,18,5,9,test,JFK2_540_32203_112020,True,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2,341,16,4,350,-2.449294e-16,1.0,0.974928,-0.222521,-2.449294e-16,1.0,-0.433884,-0.900969
3,1,JFK2,509,10609,26126,2022-12-08,2022-12-10,AFT Lite,18,18,18,6,2,test,JFK2_509_10609_26126,True,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,3,342,10,5,344,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521
4,1,JFK2,509,8427,26144,2022-12-29,2022-12-31,AFT Lite,0,36,0,1,2,test,JFK2_509_8427_26144,True,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,3,363,31,5,365,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521


In [29]:

vc_cols = ['delivered_day', 'delivered_dayofweek','order_day', 'order_dayofweek']

for col in vc_cols:
    temp_dict = df_unique[col].value_counts(normalize=True).to_dict()
    df_unique[f"{col}_VC"] = df_unique[col].map(temp_dict)
    df_test[f"{col}_VC"] = df_test[col].map(temp_dict)
    df_test_n[f"{col}_VC"] = df_test_n[col].map(temp_dict)
    


In [30]:
req_cols = [col for col in df_unique.columns if col not in ['region_id', 'warehouse_id', 'd_id', 'isbn_id',
                                                            'order_month_cos', 'order_month_sin','order_id',
                                                              'order_date', 'evsd', 'po_source_ind', 'tag',
                                                                'unique_id', 'row_group_ind','quantity_received', 
                                                                'consider', 'order_dayofyear', 'delivered_dayofyear',
                                                                'delivered_day', 'delivered_dayofweek','order_day', 'order_dayofweek']]


In [31]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

def new_metric(df):
    df['mae'] = df[['quantity_received', 'preds']].apply(lambda row: mae(row['quantity_received'], row['preds']), axis=1)
    final_error = df['mae'].sum()/ df['preds'].sum()
    return final_error

In [32]:
# StandardScaler on Target Varianle and quantity columns

from sklearn import preprocessing


scaler_t = preprocessing.StandardScaler()
df_unique['quantity_received'] = scaler_t.fit_transform(df_unique['quantity_received'].values.reshape(-1, 1))
df_test['quantity_received'] = scaler_t.transform(df_test['quantity_received'].values.reshape(-1, 1))
df_test_n['quantity_received'] = scaler_t.transform(df_test_n['quantity_received'].values.reshape(-1, 1))

scaler_s = preprocessing.StandardScaler()
df_unique['quantity_ordered'] = scaler_s.fit_transform(df_unique['quantity_ordered'].values.reshape(-1, 1))
df_unique['quantity_submitted'] = scaler_s.fit_transform(df_unique['quantity_submitted'].values.reshape(-1, 1))
df_test['quantity_ordered'] = scaler_s.transform(df_test['quantity_ordered'].values.reshape(-1, 1))
df_test_n['quantity_ordered'] = scaler_s.transform(df_test_n['quantity_ordered'].values.reshape(-1, 1))
df_test['quantity_submitted'] = scaler_s.transform(df_test['quantity_submitted'].values.reshape(-1, 1))
df_test_n['quantity_submitted'] = scaler_s.transform(df_test_n['quantity_submitted'].values.reshape(-1, 1))


In [33]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb

# Instantiate the transformer
qt = QuantileTransformer()



In [34]:
df_unique[req_cols].head(2)

,quantity_ordered,quantity_submitted,visibility,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos,delivered_day_VC,delivered_dayofweek_VC,order_day_VC,order_dayofweek_VC
0,-0.238871,-0.312479,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.781831,0.62349,-0.5,0.866025,0.974928,-0.222521,0.017824,0.165684,0.019543,0.182823
1,0.064740,-0.025241,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.781831,0.62349,-0.5,0.866025,0.974928,-0.222521,0.032819,0.165684,0.023297,0.182823


In [37]:
del model

In [48]:
import tensorflow as tf

# Define the model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(32, activation='relu', input_shape=(len(req_cols),)),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1)
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(0.001)

model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', 'mse'])

x_train = df_unique[req_cols]
y_train = df_unique['quantity_received'] 

x_test = df_test_n[req_cols]
y_test = df_test_n['quantity_received']   

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32,
                     validation_split = 0.2)

# Evaluate the model
loss = model.evaluate(x_test, y_test, verbose=2)

# print("Mean Absolute Error:", mae)


Epoch 1/10
1353/1353 [==============================] - 5s 3ms/step - loss: 0.2010 - mae: 0.1914 - mse: 0.2010 - val_loss: 0.1322 - val_mae: 0.1656 - val_mse: 0.1322
Epoch 2/10
1353/1353 [==============================] - 3s 2ms/step - loss: 0.1540 - mae: 0.1547 - mse: 0.1540 - val_loss: 0.1103 - val_mae: 0.1391 - val_mse: 0.1103
Epoch 3/10
1353/1353 [==============================] - 4s 3ms/step - loss: 0.1456 - mae: 0.1522 - mse: 0.1456 - val_loss: 0.0944 - val_mae: 0.1405 - val_mse: 0.0944
Epoch 4/10
1353/1353 [==============================] - 4s 3ms/step - loss: 0.1390 - mae: 0.1494 - mse: 0.1390 - val_loss: 0.0985 - val_mae: 0.1450 - val_mse: 0.0985
Epoch 5/10
1353/1353 [==============================] - 6s 4ms/step - loss: 0.1320 - mae: 0.1479 - mse: 0.1320 - val_loss: 0.0906 - val_mae: 0.1264 - val_mse: 0.0906
Epoch 6/10
1353/1353 [==============================] - 7s 5ms/step - loss: 0.1295 - mae: 0.1466 - mse: 0.1295 - val_loss: 0.0967 - val_mae: 0.1438 - val_mse: 0.0967
Epoc

In [49]:
loss

[0.7000719904899597, 0.30361777544021606, 0.7000719904899597]

In [50]:
# Performance metrics by epoch
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
0,0.200971,0.191382,0.200971,0.132165,0.165622,0.132165,0
1,0.153989,0.154684,0.153989,0.110263,0.139115,0.110263,1
2,0.145610,0.152218,0.145610,0.094442,0.140504,0.094442,2
3,0.138999,0.149396,0.138999,0.098493,0.144992,0.098493,3
4,0.131996,0.147867,0.131996,0.090621,0.126360,0.090621,4
5,0.129473,0.146637,0.129473,0.096682,0.143781,0.096682,5
6,0.131865,0.145463,0.131865,0.109020,0.140381,0.109020,6
7,0.123514,0.142696,0.123514,0.095822,0.163807,0.095822,7
8,0.125724,0.144506,0.125724,0.090072,0.136736,0.090072,8
9,0.122715,0.142628,0.122715,0.094505,0.140392,0.094505,9


In [51]:
df_test_n['preds'] = model.predict(df_test_n[req_cols])

701/701 [==============================] - 1s 1ms/step


In [52]:

# df_unique['quantity_received'] = scaler_t.fit_transform(df_unique['quantity_received'].values.reshape(-1, 1))
# df_test['quantity_received'] = scaler_t.transform(df_test['quantity_received'].values.reshape(-1, 1))



# Inverse transform the standardized data
scaler_t.inverse_transform(df_test_n['quantity_received'].values.reshape(1, -1))

array([[ 6., 12., 18., ...,  6., 36.,  0.]])

In [53]:
df_test_n['preds']

0       -0.317865
1       -0.213462
2       -0.034365
3        0.034106
4       -0.462131
           ...   
22401   -0.602208
22402    3.064383
22403   -0.606231
22404    0.651640
22405   -0.504110
Name: preds, Length: 22406, dtype: float32

In [54]:
# Scale data back
y_test_inv = scaler_t.inverse_transform(df_test_n['quantity_received'].values.reshape(1, -1))
y_pred_inv = scaler_t.inverse_transform(df_test_n['preds'].values.reshape(1, -1))

print('R2:', r2_score(y_test_inv, y_pred_inv))
print('MAE:', mean_absolute_error(y_test_inv, y_pred_inv))

R2: nan
MAE: 8.032719879718478


In [45]:
y_test_inv

array([[42.,  6., 12., ...,  4.,  0.,  0.]])

In [46]:
y_pred_inv

array([[31.842543 ,  3.0956783,  3.5975075, ...,  2.3654642, -1.8529931,
        -0.5700723]], dtype=float32)

In [47]:
new_metric(df_test)

-1.8361245667824362

In [117]:
import tensorflow as tf

# define and train your model here

# save the model
model.save('Final_TENSORFLOW_MODEL')

INFO:tensorflow:Assets written to: Final_TENSORFLOW_MODEL\assets


INFO:tensorflow:Assets written to: Final_TENSORFLOW_MODEL\assets


In [118]:
model2 = tf.keras.models.load_model('Final_TENSORFLOW_MODEL')

In [135]:
df_test['preds'] = model2.predict(df_test[req_cols])
new_metric(df_test)

KeyError: "['d_id_440', 'd_id_79', 'd_id_216', 'd_id_462', 'd_id_746', 'd_id_210', 'quantity_received_s', 'quantity_ordered_s', 'quantity_submitted_s'] not in index"

In [120]:
df_test.shape

(22406, 85)

In [ ]:
df_test.to_csv('final_scored_test_file.csv', index = False)